# **Feature Extraction Models**

In [1]:
import pandas as pd
import nltk
import gensim
from nltk.stem import WordNetLemmatizer, SnowballStemmer
from gensim.parsing.preprocessing import STOPWORDS
import numpy as np
from gensim import corpora,models
import time
import pickle
nltk.download('stopwords')
nltk.download('wordnet')
import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning)


import warnings
warnings.filterwarnings('ignore')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\niranjans3ln\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\niranjans3ln\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [2]:
# open a file, where you stored the pickled data
f = open('dataset/bow_corpus.pickle', 'rb')
bow_corpus=pickle.load(f)

file = open('dataset/dictionary.pickle', 'rb')
dictionary=pickle.load(file)

# later on, load trained model from file
lda_model =  models.LdaModel.load('dataset/lda_model.model')

EOFError: Ran out of input

In [ ]:
def lemmatize(text):
    return WordNetLemmatizer().lemmatize(text, pos='v')

def preprocess(text):
    result = []
    for token in gensim.utils.simple_preprocess(text):
        if token not in gensim.parsing.preprocessing.STOPWORDS and len(token) > 5:
            result.append(lemmatize(token))
    return result

In [ ]:
#Import all the clusters from the drive 
for c in range(10):
  exec('topic_{} = pd.read_csv("dataset/Mozilla/topic_{}.csv")'.format(c,c))
  exec("topic_{}= topic_{}.drop(columns=['Unnamed: 0'])".format(c,c))
  exec("topic_{}['Description'] = topic_{}['Description'].map(preprocess)".format(c,c))

In [ ]:
#creating a corpus for Word2Vec and FastText models
for i in range(10):
  exec('sent_{} = []'.format(i))
  exec('x= topic_{}'.format(i))
  for j in range(len(x)):
    exec('sent_{}.append(topic_{}.Description[{}])'.format(i,i,j))

In [ ]:
#Length of all the corpus
for sent in range(10):
  exec('print(len(sent_{}))'.format(sent))

In [ ]:
# !pip install glove_python

In [ ]:
from gensim.models import Word2Vec,FastText
from gensim.test.utils import get_tmpfile
from glove import Glove, Corpus

### **Word2Vec model training**

In [ ]:
for cluster in range(10):
  #Preparing parameters for model training
  exec('corpus = sent_{}'.format(cluster))
  vector_size = 100
  w = 6
  min_count = 5
  CBoW = 0
  iterations = 100

  #Training Word2Vec model for each cluster
  exec ('w2vmodel{} = Word2Vec(corpus, size=vector_size, window=w, min_count=min_count, sg=CBoW, iter=iterations)'.format(cluster))

  #Save the all the models in individual file
  exec('path = get_tmpfile("dataset/word2vec{}.model")'.format(cluster))
  exec('w2vmodel{}.save("dataset/word2vec{}.model")'.format(cluster), cluster))

### **FastText model training**

In [ ]:
for cluster in range(10):
  #Preparing parameters for model training
  exec('corpus = sent_{}'.format(cluster))
  vector_size = 100
  w = 6
  min_count = 5
  CBoW = 0
  iterations = 100

  #Training FastText model for each cluster
  exec ('ftmodel{} = FastText(corpus, size=vector_size, window=w, min_count=min_count, sg=CBoW, iter=iterations)'.format(cluster))

  #Save the all the models in individual file
  exec('path = get_tmpfile("dataset/ftmodel{}.model")'.format(cluster))
  exec('ftmodel{}.save("dataset/ftmodel{}.model")'.format(cluster, cluster))

### **GloVe model training**

In [ ]:
#Training GloVe model for each cluster
for cluster in range(10):
  vector_size = 100
  exec('glove_corpus{}=Corpus()'.format(cluster, cluster)) 
  exec('glove_corpus{}.fit(sent_{})'.format(cluster, cluster))
  exec('glove{}= Glove(no_components=vector_size, learning_rate=0.18, alpha=0.75, max_count=100, max_loss=10.0, random_state=None)'.format(cluster, cluster))
  exec('glove{}.fit(glove_corpus{}.matrix, epochs=1000, no_threads=3, verbose=True)'.format(cluster, cluster))
  exec('transformer = lambda dictionary2:glove{}.transform_paragraph(words, epochs=1000,ignore_missing=False)'.format(cluster, cluster))
  exec('glove{}.add_dictionary(glove_corpus{}.dictionary)'.format(cluster, cluster))

  #Save the all the models in individual file
  exec('path = get_tmpfile("dataset/glove{}.model")'.format(cluster))
  exec('glove{}.save("dataset/glove{}.model")'.format(cluster, cluster))